# Transportation Model Under Changing Commodity Status


### **Problem Description: Transportation Optimization with Demand Variability and Increased Fuel Prices**

A leather bag manufacturing company, operating from two factories: Factory1 and Factory2 based in Akure and Abeokuta respectively, aims to optimize its transportation plan to distribute leather travel bags to three retail stores  (Benin, Abuja, and Lagos). The company wants to minimize transportation costs while considering changes in demand variability among the stores and the impact of increased fuel prices.

**Objective Function:**
Minimize the total transportation costs while accounting for demand variability and increased fuel prices.

**Decision Variables:**
- $x(i, j)$: Number of units of leather bags transported from Factory $i$ to Store $j$.

**Parameters:**
- $D_{j}$: Adjusted demand requirement for Store $j$ based on demand variability.
- $C_{ij}$: Initial transportation cost to transport one unit from Factory $i$ to Store $j$.
- *Increased Fuel Price*: Factor by which transportation costs are increased due to higher fuel prices.
- $C_{ij}^{new}$: Updated transportation cost due to increased fuel prices for transporting one unit from Factory $i$ to Store $j$.

**Constraints:**
1. **Supply Capacity Constraints:**
   - Total units shipped from each factory must not exceed the total supply capacity of leather bags at each factory:
   
   &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; $\sum_{j} x[(i, j)] \leq $ Supply Capacity $_{i}$, for $i$ in Supply Nodes

2. **Demand Requirement Constraints:**
   - Adjusted demand requirement must be met for each retail store based on demand variability:
   
   &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; $\sum_{j} x[(i, j)] \leq D_{i}$, for $i$ in Supply Nodes

**Objective Function:**
Minimize the total transportation costs considering initial transportation costs and updated transportation costs due to increased fuel prices:

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Minimize: $\sum_{i} \sum_{j} C_{ij} \times x[(i, j)] + \sum_{i} \sum_{j} C_{ij}^{\text{new}} \times x[(i, j)]$

**Problem Status:**
The problem will be solved using linear programming techniques to find the optimal transportation plan and associated total costs for both the initial and changing cost scenarios due to increased fuel prices. This Linear Programming problem formulation aims to optimize the transportation of leather bags while considering demand variability and increased fuel prices, ensuring efficient distribution while minimizing costs.

In [25]:
from pulp import *

# Sample data (modify according to your specific problem)
supply_nodes = ['Factory_1_Akure', 'Factory_2_Abeokuta']
demand_nodes = ['Benin', 'Abuja', 'Lagos']
commodities = ['Travel Bag']

# Initial transportation costs
initial_transportation_costs = {
    ('Factory_1_Akure', 'Benin'): 50,
    ('Factory_1_Akure', 'Abuja'): 100,
    ('Factory_1_Akure', 'Lagos'): 75,
    ('Factory_2_Abeokuta', 'Benin'):75,
    ('Factory_2_Abeokuta', 'Abuja'): 110,
    ('Factory_2_Abeokuta', 'Lagos'): 25,
}

# Changing demand variability and increased fuel prices
demand_variability = {
    'Benin': 50,
    'Abuja': 100,
    'Lagos': 70,
}

increased_fuel_price = 1.6  # 60% increase in fuel prices

# Changein transportation costs
change_transportation_costs = {
    ('Factory_1_Akure', 'Benin'): 70,
    ('Factory_1_Akure', 'Abuja'): 120,
    ('Factory_1_Akure', 'Lagos'): 85,
    ('Factory_2_Abeokuta', 'Benin'):90,
    ('Factory_2_Abeokuta', 'Abuja'): 105,
    ('Factory_2_Abeokuta', 'Lagos'): 40,
}

# Create a Linear Programming problem
transport_model = LpProblem(name="Transportation_Model", sense=LpMinimize)

# Decision variables
x = LpVariable.dicts("Shipment", ((i, j) for i in supply_nodes for j in demand_nodes), lowBound=0, cat='Integer')

# Objective function: minimize transportation costs
transport_model += lpSum(initial_transportation_costs[(i, j)] * x[(i, j)] for i in supply_nodes for j in demand_nodes)

# Constraints: supply and demand balance
for i in supply_nodes:
    transport_model += lpSum(x[(i, j)] for j in demand_nodes) <= sum(demand_variability.values()) # Total supply capacity

for j in demand_nodes:
    transport_model += lpSum(x[(i, j)] for i in supply_nodes) >= demand_variability[j]   # Adjusted demand requirement


print(transport_model)
# Solve the problem with initial transportation costs
transport_model.solve()


print(f"Problem Status: {LpStatus[transport_model.status]}")

# Print the results with initial transportation costs
print("Optimal transportation plan (initial costs):")
for i in supply_nodes:
    for j in demand_nodes:
        if x[(i, j)].varValue > 0:
            print(f"Transport {x[(i, j)].varValue} units of {commodities[0]} from {i} to {j}: Cost=N{((initial_transportation_costs[i,j])*(x[(i, j)].varValue))}")
print("\nTotal Transportation Cost:", value(transport_model.objective))


# Update the transportation costs due to increased fuel prices
for (i, j), cost in change_transportation_costs.items():
    initial_transportation_costs[(i, j)] = cost * increased_fuel_price

# Update the objective function with changing transportation costs
transport_model.objective = lpSum(initial_transportation_costs[(i, j)] * x[(i, j)] for i in supply_nodes for j in demand_nodes)

# Solve the problem with changed transportation costs
transport_model.solve()

print(f"\nProblem Status: {LpStatus[transport_model.status]}")
# Print the results with changed transportation costs
print("Optimal transportation plan (changing costs due to increased fuel prices):")
for i in supply_nodes:
    for j in demand_nodes:
        if x[(i, j)].varValue > 0:
            print(f"Transport {x[(i, j)].varValue} units of {commodities[0]} from {i} to {j}: Cost=N{((initial_transportation_costs[i,j])*(x[(i, j)].varValue))}")
print("\nTotal Transportation Cost:", value(transport_model.objective))



Transportation_Model:
MINIMIZE
100*Shipment_('Factory_1_Akure',_'Abuja') + 50*Shipment_('Factory_1_Akure',_'Benin') + 75*Shipment_('Factory_1_Akure',_'Lagos') + 110*Shipment_('Factory_2_Abeokuta',_'Abuja') + 75*Shipment_('Factory_2_Abeokuta',_'Benin') + 25*Shipment_('Factory_2_Abeokuta',_'Lagos') + 0
SUBJECT TO
_C1: Shipment_('Factory_1_Akure',_'Abuja')
 + Shipment_('Factory_1_Akure',_'Benin')
 + Shipment_('Factory_1_Akure',_'Lagos') <= 220

_C2: Shipment_('Factory_2_Abeokuta',_'Abuja')
 + Shipment_('Factory_2_Abeokuta',_'Benin')
 + Shipment_('Factory_2_Abeokuta',_'Lagos') <= 220

_C3: Shipment_('Factory_1_Akure',_'Benin')
 + Shipment_('Factory_2_Abeokuta',_'Benin') >= 50

_C4: Shipment_('Factory_1_Akure',_'Abuja')
 + Shipment_('Factory_2_Abeokuta',_'Abuja') >= 100

_C5: Shipment_('Factory_1_Akure',_'Lagos')
 + Shipment_('Factory_2_Abeokuta',_'Lagos') >= 70

VARIABLES
0 <= Shipment_('Factory_1_Akure',_'Abuja') Integer
0 <= Shipment_('Factory_1_Akure',_'Benin') Integer
0 <= Shipment_('F

Here's a breakdown of the output for both scenarios:

**Initial Costs Scenario:**
- 50.0 units of Travel Bag are transported from Factory_1_Akure to Benin at a cost of N2500.0.
- 100.0 units of Travel Bag are transported from Factory_1_Akure to Abuja at a cost of N10000.0.
- 70.0 units of Travel Bag are transported from Factory_2_Abeokuta to Lagos at a cost of N1750.0.
- The total transportation cost for this scenario is N14250.0.

**Changing Costs Scenario (Due to Increased Fuel Prices):**
- 50.0 units of Travel Bag are transported from Factory_1_Akure to Benin at a cost of N5600.0.
- 100.0 units of Travel Bag are transported from Factory_2_Abeokuta to Abuja at a cost of N16800.0.
- 70.0 units of Travel Bag are transported from Factory_2_Abeokuta to Lagos at a cost of N4480.0.
- The total transportation cost for this scenario is N26880.0.

The optimization model successfully computed the optimal transportation plans and costs for both scenarios, considering both initial costs and the impact of increased fuel prices. The "Optimal" status indicates that the model has found the best solution based on the given inputs and constraints.